# Ride Demand Forecasting (OOT)

This notebook provides functions to run both the LSTM and the XGBoost model on each out-of-time window and export them to a single dataset.

____

In [240]:
import pandas as pd
import numpy as np
import pytz
from pandas.tseries.holiday import USFederalHolidayCalendar
from datetime import datetime
import scipy.stats as stats
pd.options.mode.chained_assignment = None  # default='warn'

import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 18, 6

import tensorflow as tf
from tensorflow import keras
from xgboost import plot_importance, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor

In [241]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [242]:
master_weather_data = pd.read_csv('../data/master_weather_data.csv')
master_weather_data = master_weather_data.drop('Unnamed: 0', axis = 1)
master_weather_data['weather_zip'] = master_weather_data['weather_zip'].astype(str)
master_weather_data['date_time'] = pd.to_datetime(master_weather_data['date_time'], utc = True)

In [243]:
df = pd.read_csv('../data/with_weather.csv')
df = df.drop_duplicates('RIDE_ID')
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis = 1)

/Users/tanmaybansal/Desktop/projectx_final/projectx_venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [244]:
df['started_on'] = pd.to_datetime(df['started_on'], utc = True)
df['started_on'] = pd.DatetimeIndex(df['started_on'])
df['started_on_hour'] = df['started_on'].apply(lambda x: pd.to_datetime(
    datetime.combine(x.date(), datetime.now().replace(microsecond=0,second=0,minute=0,hour=x.hour).time()),
    utc = True
))
df['day_of_week'] = df['started_on'].apply(lambda d: d.weekday())
df['weekend'] = df['day_of_week'].apply(lambda d: 1 if d in {5, 6} else 0)
federal_holidays = USFederalHolidayCalendar().holidays(
    start = np.min(df['started_on']),
    end = np.max(df['started_on'])
)
federal_holidays = set(pd.Series(federal_holidays).apply(lambda d: d.date()))
df['federal_holiday'] = df['started_on'].apply(lambda d: 1 if d.date() in federal_holidays else 0)

In [247]:
np.mean(master_master_df['xgboost_pred'] - master_master_df['num_rides'])

-5.1357822386223395

In [246]:
def get_hourly_data_for_zone(df, weather_data, zone_label, end_time):
    data = df[df['zone_label'] == zone_label]
    
    data = data[[
        'started_on_hour',
        'RIDE_ID',
        'day_of_week',
        'weekend',
        'HeatIndexC',
        'precipMM',
        'humidity',
        'federal_holiday',
        'zipcode'
        ]].set_index('started_on_hour').resample('H').agg({
        'RIDE_ID':'count',
        'day_of_week':'max',
        'weekend':'max',
        'HeatIndexC': 'mean',
        'precipMM': 'mean',
        'humidity': 'mean',
        'federal_holiday': 'max'
    }).rename(
        columns = {
            'RIDE_ID': 'num_rides'
        }
    ).reset_index('started_on_hour')
    
    complete_data = data[~(pd.isna(data['day_of_week']))]
    missing_data = data[pd.isna(data['day_of_week'])]
    
    # For NaN values (i.e. where total hourly rides are 0)
    missing_data['day_of_week'] = missing_data['started_on_hour'].apply(lambda d: d.weekday())
    missing_data['weekend'] = missing_data['day_of_week'].apply(lambda d: 1 if d in {5, 6} else 0)
    federal_holidays = USFederalHolidayCalendar().holidays(
        start = np.min(missing_data['started_on_hour']),
        end = np.max(missing_data['started_on_hour'])
    )
    federal_holidays = set(pd.Series(federal_holidays).apply(lambda d: d.date()))
    missing_data['federal_holiday'] = missing_data['started_on_hour'].apply(lambda d: 1 if d.date() in federal_holidays else 0)
    
    missing_data['common_zipcode'] = str(stats.mode(df[df['zone_label'] == zone_label]['zipcode'])[0][0])[:5]
    weather_data = weather_data[[
        'weather_zip',
        'date_time',
        'HeatIndexC',
        'precipMM',
        'humidity'
    ]]
    missing_data = pd.merge(
        missing_data.drop(
            ['HeatIndexC', 'precipMM', 'humidity'], axis = 1
            ), 
        weather_data,
        how = 'left',
        left_on = ['common_zipcode', 'started_on_hour'],
        right_on = ['weather_zip', 'date_time']
    ).drop(['weather_zip', 'date_time', 'common_zipcode'], axis = 1)
    
    zone_data = pd.concat([
        complete_data,
        missing_data
    ]).sort_values('started_on_hour').reset_index(drop = True)
    
    hourly_count = zone_data.set_index('started_on_hour')
    
    hourly_count = pd.concat([
        hourly_count,
        pd.get_dummies(hourly_count['day_of_week'], prefix='day_of_week')
    ], axis = 1).drop('day_of_week', axis = 1)

    hourly_count = hourly_count[(hourly_count.index >= pd.to_datetime('2016-06-01', utc = True)) &
                 (hourly_count.index <= pd.to_datetime(end_time, utc = True))]

    hourly_count['HeatIndexC'] = hourly_count['HeatIndexC'].replace(np.nan, np.mean(hourly_count['HeatIndexC']))
    hourly_count['precipMM'] = hourly_count['precipMM'].replace(np.nan, np.mean(hourly_count['precipMM']))
    hourly_count['humidity'] = hourly_count['humidity'].replace(np.nan, np.mean(hourly_count['humidity']))
    
    return hourly_count

In [248]:
def get_xgboost_pred(hourly_count, test_threshold):
    hourly_count['hour'] = hourly_count.index.hour
    hourly_count['day_of_month'] = hourly_count.index.day

    X = hourly_count.drop('num_rides', axis = 1)
    y = hourly_count['num_rides']
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=RANDOM_SEED, shuffle = False)
    
    X_train = X[
    (X.index <= pd.to_datetime(test_threshold, utc = True))
    ]
    X_test = X[
    (X.index > pd.to_datetime(test_threshold, utc = True))
    ]
    y_train = y[
        (y.index <= pd.to_datetime(test_threshold, utc = True))
    ]
    y_test = y[
        (y.index > pd.to_datetime(test_threshold, utc = True))
    ]
    
    reg = xgb.XGBRegressor(n_estimators=1000)
    reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
       verbose=False)
    y_pred = reg.predict(X_test)
    
    return y_pred

In [123]:
def get_lstm_pred(hourly_count, test_threshold):
    train_size = len(hourly_count[hourly_count.index <= pd.to_datetime(test_threshold, utc = True)])
    test_size = len(hourly_count) - train_size
    
    num_rides = hourly_count['num_rides']
    hourly_count = pd.DataFrame(MinMaxScaler(feature_range=(0, 1)).fit_transform(hourly_count))
    hourly_count[0] = num_rides.reset_index(drop = True).values
    
    train, test = hourly_count.iloc[0:train_size], hourly_count.iloc[train_size:len(hourly_count)]

    time_steps = 6

    def create_dataset(X, y, time_steps=1):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            v = X.iloc[i:(i + time_steps)].values
            Xs.append(v)
            ys.append(y.iloc[i + time_steps])
        return np.array(Xs), np.array(ys)

    X_train, y_train = create_dataset(train, train[0], time_steps)
    X_test, y_test = create_dataset(test, test[0], time_steps)

    
    model = keras.Sequential()

    model.add(keras.layers.LSTM(
      units=128,
      input_shape=(X_train.shape[1], X_train.shape[2]),
      activation="relu",
      return_sequences=True
    ))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(units=1))

    callbacks = [
        keras.callbacks.ReduceLROnPlateau(
            monitor = 'val_loss', factor = 0.05, patience = 10, min_lr = 0.0001
        ),
        keras.callbacks.EarlyStopping(
            monitor = 'val_loss', patience = 15, verbose = 1
        )
    ]
    model.compile(
      loss='mean_squared_error',
      optimizer=keras.optimizers.Adam(0.001, clipvalue=0.5),
        metrics = [
            'MeanSquaredError',
            'RootMeanSquaredError',
            'MeanAbsoluteError'
        ]
    )
    
    history = model.fit(
    X_train, y_train,
    epochs=45,
    batch_size=32,
    callbacks=callbacks,
    validation_split=0.1,
    verbose=0,
    shuffle=False
    )
    
    y_pred = model.predict(X_test)
    
    return y_pred

In [204]:
zone = 0
time_steps = 6
test_threshold = '2016-01-31 23:00'
end_time = '2017-02-28 23:00'

In [205]:
hourly_count = get_hourly_data_for_zone(df, master_weather_data, zone, end_time)
xgboost_pred = get_xgboost_pred(hourly_count, test_threshold)
lstm_pred = get_lstm_pred(hourly_count, test_threshold)

[18:24:47] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:59: Label set is empty.
[18:24:47] WARNING: /Users/travis/build/dmlc/xgboost/src/metric/elementwise_metric.cu:336: label set is empty
[18:24:47] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:59: Label set is empty.
[18:24:47] WARNING: /Users/travis/build/dmlc/xgboost/src/metric/elementwise_metric.cu:336: label set is empty
[18:24:47] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:59: Label set is empty.
[18:24:47] WARNING: /Users/travis/build/dmlc/xgboost/src/metric/elementwise_metric.cu:336: label set is empty
[18:24:47] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:59: Label set is empty.
[18:24:47] WARNING: /Users/travis/build/dmlc/xgboost/src/metric/elementwise_metric.cu:336: label set is empty
[18:24:47] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:59: Label set is empty.
[18:24:47] WARN

IndexError: tuple index out of range

___

We also developed a stacked model, i.e. a Random Forest Regression on top of predictions from the XGBoost and the LSTM model. However, we finally went for the simpler model.

In [198]:
master_df = hourly_count[hourly_count.index > pd.to_datetime(test_threshold, utc = True)].iloc[time_steps:]
master_df['xgboost_pred'] = xgboost_pred[time_steps : ]
master_df['lstm_pred'] = lstm_pred

X = master_df.drop('num_rides', axis = 1)
y = master_df['num_rides']

X_1, X_2, y_1, y_2 = train_test_split(X, y, test_size=0.5, random_state=RANDOM_SEED, shuffle = False)

model1 = RandomForestRegressor(random_state = RANDOM_SEED).fit(X_1, y_1)
model2 = RandomForestRegressor(random_state = RANDOM_SEED).fit(X_2, y_2)

master_df['stacked_pred'] = np.concatenate((model2.predict(X_1), model1.predict(X_2)))
master_df['zone'] = zone
print(np.mean(master_df['num_rides'] - master_df['stacked_pred']))

-4.553617886178861


In [199]:
master_df.head(2)

num_rides  weekend  HeatIndexC  precipMM  \
started_on_hour                                                       
2017-01-01 06:00:00+00:00        111      1.0        11.0       0.0   
2017-01-01 07:00:00+00:00        115      1.0        11.0       0.0   

                            humidity  federal_holiday  day_of_week_0.0  \
started_on_hour                                                          
2017-01-01 06:00:00+00:00  69.873874              0.0                0   
2017-01-01 07:00:00+00:00  65.886957              0.0                0   

                           day_of_week_1.0  day_of_week_2.0  day_of_week_3.0  \
started_on_hour                                                                
2017-01-01 06:00:00+00:00                0                0                0   
2017-01-01 07:00:00+00:00                0                0                0   

                           day_of_week_4.0  day_of_week_5.0  day_of_week_6.0  \
started_on_hour                                                                
2017-01-01 06:00:00+00:00                0                0                1   
2017-01-01 07:00:00+00:00                0                0                1   

                           hour  day_of_month  xgboost_pred   lstm_pred  \
started_on_hour                                                           
2017-01-01 06:00:00+00:00     6             1    126.842628  126.620911   
2017-01-01 07:00:00+00:00     7             1     66.242210   77.290909   

                           stacked_pred  zone  
started_on_hour                                
2017-01-01 06:00:00+00:00        168.49     0  
2017-01-01 07:00:00+00:00         92.69     0

In [200]:
# master_master_df = pd.concat([master_master_df, master_df])
# master_master_df.to_csv('data/all_ride_predictions.csv')

In [263]:
# master_master_df = pd.DataFrame()

In [271]:
# oot windows
windows = [
    ['2016-06-30 23:00', '2016-07-31 23:00'],
    ['2016-07-31 23:00', '2016-08-31 23:00'],
    ['2016-08-31 23:00', '2016-09-30 23:00'],
    ['2016-09-30 23:00', '2016-10-31 23:00'],
    ['2016-10-31 23:00', '2016-11-30 23:00'],
    ['2016-11-30 23:00', '2016-12-31 23:00'],
    ['2016-12-31 23:00', '2017-01-31 23:00']
]

In [272]:
for zone in range(0, 10):
    for window in windows:
        print(zone, ':', window[0], '-', window[1])
        time_steps = 6
        test_threshold = window[0]
        end_time = window[1]

        hourly_count = get_hourly_data_for_zone(df, master_weather_data, zone, end_time)
        xgboost_pred = get_xgboost_pred(hourly_count, test_threshold)
        lstm_pred = get_lstm_pred(hourly_count, test_threshold)

        master_df = hourly_count[hourly_count.index > pd.to_datetime(test_threshold, utc = True)].iloc[time_steps:]
        master_df['xgboost_pred'] = xgboost_pred[time_steps : ]
        master_df['lstm_pred'] = lstm_pred

        X = master_df.drop('num_rides', axis = 1)
        y = master_df['num_rides']

        X_1, X_2, y_1, y_2 = train_test_split(X, y, test_size=0.5, random_state=RANDOM_SEED, shuffle = False)

        model1 = RandomForestRegressor(random_state = RANDOM_SEED).fit(X_1, y_1)
        model2 = RandomForestRegressor(random_state = RANDOM_SEED).fit(X_2, y_2)

        master_df['stacked_pred'] = np.concatenate((model2.predict(X_1), model1.predict(X_2)))
        master_df['zone'] = zone

        master_master_df = pd.concat([master_master_df, master_df])
        master_master_df.to_csv('data/all_ride_predictions_2.csv')

9 : 2016-07-31 23:00 - 2016-08-31 23:00
Epoch 00020: early stopping
9 : 2016-08-31 23:00 - 2016-09-30 23:00
Epoch 00016: early stopping
9 : 2016-09-30 23:00 - 2016-10-31 23:00
9 : 2016-10-31 23:00 - 2016-11-30 23:00
Epoch 00016: early stopping
9 : 2016-11-30 23:00 - 2016-12-31 23:00
Epoch 00033: early stopping
9 : 2016-12-31 23:00 - 2017-01-31 23:00


In [273]:
master_master_df

num_rides  weekend  HeatIndexC  precipMM  \
started_on_hour                                                       
2016-07-01 06:00:00+00:00          2      0.0        28.0       0.0   
2016-07-01 07:00:00+00:00          0      0.0        30.0       0.0   
2016-07-01 08:00:00+00:00          0      0.0        32.0       0.0   
2016-07-01 09:00:00+00:00          0      0.0        34.0       0.0   
2016-07-01 10:00:00+00:00          0      0.0        35.0       0.0   
...                              ...      ...         ...       ...   
2017-01-31 19:00:00+00:00          6      0.0        16.0       0.0   
2017-01-31 20:00:00+00:00          8      0.0        15.0       0.0   
2017-01-31 21:00:00+00:00          6      0.0        15.0       0.0   
2017-01-31 22:00:00+00:00          6      0.0        14.0       0.0   
2017-01-31 23:00:00+00:00          7      0.0        14.0       0.0   

                            humidity  federal_holiday  day_of_week_0.0  \
started_on_hour                                                          
2016-07-01 06:00:00+00:00  94.000000              0.0                0   
2016-07-01 07:00:00+00:00  85.000000              0.0                0   
2016-07-01 08:00:00+00:00  75.000000              0.0                0   
2016-07-01 09:00:00+00:00  66.000000              0.0                0   
2016-07-01 10:00:00+00:00  62.000000              0.0                0   
...                              ...              ...              ...   
2017-01-31 19:00:00+00:00  58.000000              0.0                0   
2017-01-31 20:00:00+00:00  64.125000              0.0                0   
2017-01-31 21:00:00+00:00  67.000000              0.0                0   
2017-01-31 22:00:00+00:00  75.666667              0.0                0   
2017-01-31 23:00:00+00:00  84.857143              0.0                0   

                           day_of_week_1.0  day_of_week_2.0  day_of_week_3.0  \
started_on_hour                                                                
2016-07-01 06:00:00+00:00                0                0                0   
2016-07-01 07:00:00+00:00                0                0                0   
2016-07-01 08:00:00+00:00                0                0                0   
2016-07-01 09:00:00+00:00                0                0                0   
2016-07-01 10:00:00+00:00                0                0                0   
...                                    ...              ...              ...   
2017-01-31 19:00:00+00:00                1                0                0   
2017-01-31 20:00:00+00:00                1                0                0   
2017-01-31 21:00:00+00:00                1                0                0   
2017-01-31 22:00:00+00:00                1                0                0   
2017-01-31 23:00:00+00:00                1                0                0   

                           day_of_week_4.0  day_of_week_5.0  day_of_week_6.0  \
started_on_hour                                                                
2016-07-01 06:00:00+00:00                1                0                0   
2016-07-01 07:00:00+00:00                1                0                0   
2016-07-01 08:00:00+00:00                1                0                0   
2016-07-01 09:00:00+00:00                1                0                0   
2016-07-01 10:00:00+00:00                1                0                0   
...                                    ...              ...              ...   
2017-01-31 19:00:00+00:00                0                0                0   
2017-01-31 20:00:00+00:00                0                0                0   
2017-01-31 21:00:00+00:00                0                0                0   
2017-01-31 22:00:00+00:00                0                0                0   
2017-01-31 23:00:00+00:00                0                0                0   

                           hour  day_of_month  xgboost_pre

_____

In [274]:
np.mean(master_master_df['stacked_pred'] - master_master_df['num_rides'])

0.71727013203283

___